In [ ]:
import sys

sys.path.append("..")

from core.DataLoader import DataPreprocessor, get_load_config_from_yaml
from importlib import reload
import matplotlib.pyplot as plt
import os


if False:
    PLOTS_DIR = "plots/ToponiumInfusedTransformer/"
    MODEL_DIR = "models/mixed_model"
elif True:
    PLOTS_DIR = "plots/RegressionTransformer_nu_flows_neutrino_only/"
    MODEL_DIR = "../models/model_8874707/"
else:
    PLOTS_DIR = "plots/RegressionTransforsmer_/"
    MODEL_DIR = "../models/regression_transformer_old/"


CONFIG_PATH = "../config/workspace_config.yaml"

# Create directories
os.makedirs(PLOTS_DIR, exist_ok=True)

# Set plotting style
plt.rcParams.update({"font.size": 18})

# Load data
load_config = get_load_config_from_yaml(CONFIG_PATH)
DataProcessor = DataPreprocessor(load_config)
data_config = DataProcessor.load_from_npz(
    load_config.data_path["nominal"], event_numbers="odd"
)
PREDICTIONS_DIR = os.path.join(load_config.data_dir, "RegressionTransformer_nu_flows_neutrino_only", "odd_predictions/")
os.makedirs(PREDICTIONS_DIR, exist_ok=True)

X, y = DataProcessor.get_data()
del DataProcessor

In [ ]:
import core.reconstruction.BaselineMethods as BaselineMethods
import core.evaluation.reconstruction_evaluator as Evaluation
from core.reconstruction import (
    GroundTruthReconstructor,
    PerfectAssignmentReconstructor,
    CompositeNeutrinoComponentReconstructor,
)

reload(Evaluation)
reload(BaselineMethods)
ground_truth_assigner = GroundTruthReconstructor(data_config, use_nu_flows=True)
perfect_assigner = PerfectAssignmentReconstructor(
    data_config, neutrino_reco_name=r"PIL Transformer $\nu$"
)
perfect_assigner.load_model(f"{MODEL_DIR}/odd_model.keras")
composite_neutrino_predictor = CompositeNeutrinoComponentReconstructor(
    data_config, use_nu_flows=True, axis=(-1), name=r"PIL Transformer $\nu$"
)
composite_neutrino_predictor.load_model(f"{MODEL_DIR}/odd_model.keras")

prediction_manager = Evaluation.PredictionManager(
    [ground_truth_assigner, perfect_assigner,composite_neutrino_predictor], X, y
)


prediction_manager.save_predictions(PREDICTIONS_DIR)